In [1]:
# $ 0 u l $ h i f t e r
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.DataFrame(index = None, columns = ['path', 'category'])
rootdir = '../dataset/train'
test_path = '../dataset/test_stg1/'
filelist = []
category = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        temp = file.split('/')
        filelist.append(os.path.join(subdir, file))
filelist = filelist[1:]
for i, file in enumerate(filelist):
    temp = filelist[i].split('\\')
    category.append(temp[-2])
filelist = [file.replace("\\", "/", 2) for file in filelist]
df['path'] = filelist
df['category'] = category
df.head()

,path,category
0,../dataset/train/ALB/img_00003.jpg,ALB
1,../dataset/train/ALB/img_00010.jpg,ALB
2,../dataset/train/ALB/img_00012.jpg,ALB
3,../dataset/train/ALB/img_00015.jpg,ALB
4,../dataset/train/ALB/img_00019.jpg,ALB


In [3]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
unique_classes = len(np.unique(df["category"]))
lbc = LabelEncoder()
lbc = lbc.fit_transform(df["category"])
hotEnc = to_categorical(lbc, num_classes=unique_classes)
df["category"] = list(hotEnc)
df.head()

Using TensorFlow backend.


,path,category
0,../dataset/train/ALB/img_00003.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,../dataset/train/ALB/img_00010.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,../dataset/train/ALB/img_00012.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,../dataset/train/ALB/img_00015.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,../dataset/train/ALB/img_00019.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [4]:
from keras.preprocessing.image import ImageDataGenerator
DIM = (128, 128)
img_gen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False,
    height_shift_range=0.05,
    width_shift_range=0.1,
    rotation_range=5,
    shear_range=0.1,
    fill_mode="reflect",
    zoom_range=0.15
)

In [5]:
def flow_from_dataframe(img_data, df, x_col, y_col, **args):
    base_dir = os.path.dirname(df[x_col].values[0])
    df_gen = img_gen.flow_from_directory(base_dir, class_mode="sparse", **args)
    df_gen.filenames = df[x_col].values
    df_gen.classes = np.stack(df[y_col].values)
    df_gen.samples = df.shape[0]
    df_gen.n = df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = ''
    return df_gen

In [6]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.25, random_state=4)
train_df.shape
valid_df.shape

(2832, 2)

(945, 2)

In [7]:
train_gen = flow_from_dataframe(img_gen, train_df, x_col="path", y_col="category", target_size=DIM, batch_size=128)
valid_gen = flow_from_dataframe(img_gen, valid_df, x_col="path", y_col="category", target_size=DIM, batch_size=256)
test_x, test_y = next(flow_from_dataframe(img_gen, valid_df, x_col="path", y_col="category", target_size=DIM, batch_size=1024))

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [8]:
train_x, train_y = next(train_gen)
train_x.shape
train_y.shape
img_dim = train_x.shape[1:]
img_dim

(128, 128, 128, 3)

(128, 8)

(128, 128, 3)

In [9]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import fbeta_score

In [10]:
inp_tens = Input(shape=img_dim)
base_model = VGG16(include_top=False, input_shape=img_dim)
bn = BatchNormalization()(inp_tens)
x = base_model(bn)
x = Flatten()(x)
output = Dense(unique_classes, activation="softmax")(x)
model = Model(inp_tens, output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 65544     
Total params: 14,780,244
Trainable params: 14,780,238
Non-trainable params: 6
_________________________________________________________________


In [11]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
from keras.optimizers import Adam

In [12]:
history = History()
callbacks = [history, EarlyStopping(monitor="val_loss", patience=3, verbose=1, min_delta=1e-4), ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=1,
                                                                                                                 cooldown=0, min_lr=1e-7, verbose=1),
            ModelCheckpoint(filepath="weights.best.hd5", verbose=1, save_best_only=True, save_weights_only=True, mode="auto")]

In [13]:
model.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

In [14]:
batch_size = 512
steps_per_epoch = len(train_df) / batch_size
steps_per_epoch

5.53125

In [15]:
# model.fit_generator(train_gen, steps_per_epoch=steps_per_epoch, validation_data=(test_x, test_y), epochs=15, callbacks=callbacks, use_multiprocessing=True, workers=-1)

In [16]:
model.load_weights("weights.best.hd5")

In [17]:
from glob import glob
test_files_path = glob(test_path + "*jpg", recursive=True)
test_files_path[:5]
submissionDf = pd.DataFrame(index=None, columns=["path", "image"])
submissionDf["path"] = test_files_path
test_files_path = [x.replace("\\", "/", 2) for x in test_files_path]
path_split = [x.split("/") for x in test_files_path]
image_name = [x[-1] for x in path_split]
submissionDf["path"] = test_files_path
submissionDf["image"] = image_name
submissionDf.head()

['../dataset/test_stg1\\img_00005.jpg',
 '../dataset/test_stg1\\img_00007.jpg',
 '../dataset/test_stg1\\img_00009.jpg',
 '../dataset/test_stg1\\img_00018.jpg',
 '../dataset/test_stg1\\img_00027.jpg']

,path,image
0,../dataset/test_stg1/img_00005.jpg,img_00005.jpg
1,../dataset/test_stg1/img_00007.jpg,img_00007.jpg
2,../dataset/test_stg1/img_00009.jpg,img_00009.jpg
3,../dataset/test_stg1/img_00018.jpg,img_00018.jpg
4,../dataset/test_stg1/img_00027.jpg,img_00027.jpg


In [18]:
test_gen = flow_from_dataframe(img_gen, submissionDf, x_col="path", y_col="image", target_size=DIM, batch_size=256)

Found 0 images belonging to 0 classes.


In [19]:
pred = model.predict_generator(test_gen, verbose=1, steps=4)

4/4 [==============================] - 210s 52s/step


In [21]:
len(pred)

1000

In [18]:
unique_labels = np.unique(category)

In [20]:
lbc = LabelEncoder()
lbc.fit_transform(unique_labels)
# for i, label in enumerate(list(unique_labels)):
#     (i, label)
#     submissionDf[label] = pred[:,i]

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [23]:
submissionDf.head()
# submissionDf.drop("label", axis=1, inplace=True)
# submissionDf.head()

,path,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,../dataset/test_stg1/img_00005.jpg,img_00005.jpg,0.741991,0.063520,0.016341,0.006725,0.026776,0.034861,0.005176,0.104610
1,../dataset/test_stg1/img_00007.jpg,img_00007.jpg,0.054553,0.028322,0.021593,0.005758,0.846595,0.016415,0.012608,0.014156
2,../dataset/test_stg1/img_00009.jpg,img_00009.jpg,0.359687,0.038134,0.044493,0.016797,0.047116,0.107960,0.134772,0.251040
3,../dataset/test_stg1/img_00018.jpg,img_00018.jpg,0.563564,0.040681,0.034278,0.022685,0.148841,0.083026,0.024843,0.082081
4,../dataset/test_stg1/img_00027.jpg,img_00027.jpg,0.665456,0.039488,0.035409,0.016531,0.067312,0.108548,0.010110,0.057145


In [24]:
submissionDf.to_csv("predictions_test_stage_1.csv")

In [25]:
temp = submissionDf.iloc[:,2:]
temp = temp.idxmax(axis = 1)
submissionDf["PREDICTED"] = temp
submissionDf.head()

,path,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT,PREDICTED
0,../dataset/test_stg1/img_00005.jpg,img_00005.jpg,0.741991,0.063520,0.016341,0.006725,0.026776,0.034861,0.005176,0.104610,ALB
1,../dataset/test_stg1/img_00007.jpg,img_00007.jpg,0.054553,0.028322,0.021593,0.005758,0.846595,0.016415,0.012608,0.014156,NoF
2,../dataset/test_stg1/img_00009.jpg,img_00009.jpg,0.359687,0.038134,0.044493,0.016797,0.047116,0.107960,0.134772,0.251040,ALB
3,../dataset/test_stg1/img_00018.jpg,img_00018.jpg,0.563564,0.040681,0.034278,0.022685,0.148841,0.083026,0.024843,0.082081,ALB
4,../dataset/test_stg1/img_00027.jpg,img_00027.jpg,0.665456,0.039488,0.035409,0.016531,0.067312,0.108548,0.010110,0.057145,ALB


In [21]:
from sklearn.metrics.classification import classification_report
lbc = LabelEncoder()
lbc.fit_transform(unique_labels)
label_map = dict(zip(lbc.classes_, range(len(lbc.classes_))))
inv_label_map = dict(zip(range(len(lbc.classes_)), lbc.classes_))
label_map
inv_label_map

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

{'ALB': 0,
 'BET': 1,
 'DOL': 2,
 'LAG': 3,
 'NoF': 4,
 'OTHER': 5,
 'SHARK': 6,
 'YFT': 7}

{0: 'ALB',
 1: 'BET',
 2: 'DOL',
 3: 'LAG',
 4: 'NoF',
 5: 'OTHER',
 6: 'SHARK',
 7: 'YFT'}

In [22]:
valid_pred = model.predict_generator(valid_gen, verbose=1, steps=4)

4/4 [==============================] - 190s 48s/step


In [23]:
ordered_cat = ["ALB", "BET", "DOL", "LAG", "NoF", "OTHER", "SHARK", "YFT"]
valid_pred_df = pd.DataFrame(valid_pred, columns=ordered_cat, index=None)
valid_pred_df.to_csv("valid_preds.csv", index=False)
valid_pred_df.head()

,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,0.281608,0.059779,0.054064,0.018561,0.117386,0.064064,0.164949,0.239589
1,0.085895,0.031697,0.028718,0.005075,0.759327,0.043788,0.017414,0.028085
2,0.193128,0.063256,0.089204,0.011810,0.108013,0.053056,0.072625,0.408909
3,0.141833,0.055699,0.037708,0.008507,0.005079,0.055148,0.009460,0.686566
4,0.487018,0.042125,0.027779,0.010026,0.031473,0.109275,0.074633,0.217672


In [24]:
len(valid_pred)

945

In [25]:
temp = valid_df["category"]
y_true = []
for val in temp:
    y_true.append(inv_label_map[np.argmax(val)])
y_true[:5]

['YFT', 'BET', 'ALB', 'YFT', 'YFT']

In [41]:
temp2 = valid_pred_df.iloc[:, :]
temp2.head()
y_pred = temp2.idxmax(axis=1)
y_pred[:5]

,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,0.281608,0.059779,0.054064,0.018561,0.117386,0.064064,0.164949,0.239589
1,0.085895,0.031697,0.028718,0.005075,0.759327,0.043788,0.017414,0.028085
2,0.193128,0.063256,0.089204,0.011810,0.108013,0.053056,0.072625,0.408909
3,0.141833,0.055699,0.037708,0.008507,0.005079,0.055148,0.009460,0.686566
4,0.487018,0.042125,0.027779,0.010026,0.031473,0.109275,0.074633,0.217672


0    ALB
1    NoF
2    YFT
3    YFT
4    ALB
dtype: object

In [45]:
report = classification_report(y_true, y_pred, target_names=ordered_cat)
print(report)

              precision    recall  f1-score   support

         ALB       0.44      0.80      0.57       424
         BET       0.00      0.00      0.00        46
         DOL       0.00      0.00      0.00        31
         LAG       0.00      0.00      0.00        13
         NoF       0.12      0.06      0.08       126
       OTHER       0.00      0.00      0.00        76
       SHARK       0.00      0.00      0.00        46
         YFT       0.16      0.09      0.12       183

   micro avg       0.39      0.39      0.39       945
   macro avg       0.09      0.12      0.10       945
weighted avg       0.25      0.39      0.29       945



In [47]:
print(report, file=open("benchmarks.txt", "w"))